In [ ]:
!nvidia-smi  # Executa o utilitário NVIDIA System Management Interface (nvidia-smi)
             # Esse comando exibe informações sobre as GPUs NVIDIA disponíveis no sistema,
             # incluindo:
             # - Modelo da GPU
             # - Versão do driver NVIDIA
             # - Utilização da GPU (memória, processamento)
             # - Processos que estão usando a GPU
             #
             # É frequentemente usado em ambientes como Google Colab, Jupyter Notebooks ou
             # servidores remotos para confirmar:
             # - Se existe uma GPU disponível
             # - Se ela está corretamente configurada
             # - Se há recursos suficientes (ex: memória) antes de treinar modelos
             #
             # Caso não haja GPU instalada ou reconhecida, geralmente o comando
             # retornará erro ou mostrará "No devices were found".

In [ ]:
# Instala as bibliotecas necessárias para trabalhar com modelos de NLP da Hugging Face

!pip install transformers   # Instala a biblioteca Transformers, que fornece:
                            # - Modelos pré-treinados (BERT, GPT, T5, etc.)
                            # - Pipelines prontos (tradução, resumo, classificação, etc.)
                            # - Ferramentas para fine-tuning e inferência

!pip install accelerate     # Instala a biblioteca Accelerate, usada para:
                            # - Acelerar treinos em GPU/TPU
                            # - Facilitar treinos distribuídos (multi-GPU ou multi-nó)
                            # - Otimizar performance sem mudar o código do modelo

In [ ]:
from google.colab import files
import os

# Nome esperado para o arquivo de corpus que será utilizado no treinamento ou processamento NLP
NOME_ARQUIVO_CORPUS = "corpus.txt"

# Verifica se o arquivo já existe no diretório atual (evita upload duplicado)
if not os.path.exists(NOME_ARQUIVO_CORPUS):

  # Solicita ao usuário o upload manual do arquivo
  print(f"Fazendo upload do arquivo '{NOME_ARQUIVO_CORPUS}'...")
  print("Por favor, selecione o seu arquivo de corpus.")

  # Abre o seletor de arquivos do Google Colab para upload
  uploaded = files.upload()

  # Verifica se o arquivo enviado tem exatamente o nome esperado
  if NOME_ARQUIVO_CORPUS in uploaded:
    print(f"\nArquivo '{NOME_ARQUIVO_CORPUS}' carregado com sucesso!")
  else:
    # Se o nome for diferente, o script alerta o usuário
    print(f"\nERRO: O arquivo carregado não se chama '{NOME_ARQUIVO_CORPUS}'.")
else:
  # Caso o arquivo já exista, evita novo upload
  print(f"Arquivo '{NOME_ARQUIVO_CORPUS}' já existe no ambiente.")


In [ ]:
import torch
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    TextDataset,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from sklearn.model_selection import train_test_split


# Modelo base pré-treinado em português — ótimo ponto de partida para fine-tuning.
# Este modelo já passou por treinamento com textos portugueses, o que acelera o aprendizado.
MODELO_BASE = "pierreguillou/gpt2-small-portuguese"

# Nome do arquivo de corpus que será usado para treinar o modelo.
# Ele deve estar no diretório de trabalho atual.
ARQUIVO_CORPUS = "corpus.txt"

# Diretório onde o modelo fine-tunado será salvo após o treinamento.
PASTA_SAIDA = "./modelo_gpt2"


print(f"Carregando Tokenizador do modelo base: {MODELO_BASE}...")
# O tokenizador converte texto puro em tokens numéricos (IDs) entendidos pelo modelo.
# Cada token representa uma palavra, subpalavra ou símbolo.
tokenizer = GPT2Tokenizer.from_pretrained(MODELO_BASE)

print(f"Carregando o Modelo base: {MODELO_BASE}...")
# GPT2LMHeadModel é uma variante do GPT-2 voltada para tarefas de modelagem de linguagem (Language Modeling),
# ou seja, prever o próximo token em uma sequência — ideal para geração de texto.
model = GPT2LMHeadModel.from_pretrained(MODELO_BASE)

print("Tokenizador e Modelo carregados com sucesso!")


In [ ]:
# Vamos dividir o corpus manualmente para garantir que o modelo
# seja avaliado em dados nunca vistos durante o treino (evita overfitting).
print("✂️  Dividindo o corpus em Treino (90%) e Validação (10%)...")

# Lê o arquivo completo do corpus como uma única string
with open(ARQUIVO_CORPUS, "r", encoding="utf-8") as f:
    texto_completo = f.read()

# Calcula o ponto de corte baseado em 90% do tamanho total do texto
tamanho_corte = int(len(texto_completo) * 0.9)

# Separa o texto em duas partes:
# - Treinamento: 90%
# - Validação: 10%
texto_treino = texto_completo[:tamanho_corte]
texto_validacao = texto_completo[tamanho_corte:]

# Salva os splits como arquivos temporários
with open("train_split.txt", "w", encoding="utf-8") as f:
    f.write(texto_treino)

with open("val_split.txt", "w", encoding="utf-8") as f:
    f.write(texto_validacao)

# Cria datasets tokenizados a partir dos arquivos, com blocos fixos
# block_size define o tamanho de sequência que o GPT-2 vai "ver" por vez
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="train_split.txt",
    block_size=128
)

eval_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="val_split.txt",
    block_size=128
)

print(f"   Blocos de Treino: {len(train_dataset)} | Blocos de Validação: {len(eval_dataset)}")

# O DataCollator agrupa tokens em batches para o modelo,
# aplicando máscaras ou shifts se necessário.
# mlm=False indica que NÃO é Masked Language Modeling (como BERT),
# mas um modelo casual (autoregressivo), como o GPT.
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # GPT prevê o próximo token, não preenche máscaras
)

In [ ]:
print("Configurando os parâmetros de treinamento...")

# 'TrainingArguments' define os hiperparâmetros e comportamentos do treinamento.
training_args = TrainingArguments(
    # Onde o modelo e checkpoints serão salvos
    output_dir=PASTA_SAIDA,
    # Se a pasta já existir, sobrescreve sem perguntar
    overwrite_output_dir=True,

    # Número total de épocas (passadas completas pelo dataset)
    num_train_epochs=15,

    # Tamanho do batch por GPU/TPU/CPU (aqui, 4 exemplos por passo)
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,

    # Gradiente é acumulado por 4 passos antes do update dos pesos
    # — isso simula um batch maior (4 × 4 = 16) sem estourar memória)
    gradient_accumulation_steps=4,

    # Taxa de aprendizado inicial
    learning_rate=8e-5,
    # Número de passos antes de atingir a LR máxima (esquenta)
    warmup_steps=300,

    # Esquema de decaimento da LR
    lr_scheduler_type="linear",

    # Regularização simples (evita overfitting — estamos desativando)
    weight_decay=0.0,

    # Limita o número de checkpoints salvos para economizar espaço
    save_total_limit=2,

    # Frequência e forma de avaliação
    eval_strategy="steps",     # avaliar ao longo do treino
    eval_steps=400,            # avalia a cada 400 passos
    save_steps=400,            # salva modelo a cada 400 passos
    logging_steps=50,          # loga métricas a cada 50 passos

    # Carrega automaticamente o melhor modelo após o treino
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,   # menor loss é melhor

    # Usa precisão mista (float16) — acelera em GPUs compatíveis
    fp16=True,

    # Evita o HuggingFace enviar logs para W&B
    report_to="none"
)

# O Trainer é o orquestrador do processo:
# conecta o modelo, datasets, collator, args e executa o treino/validação.
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Passamos também o conjunto de validação
)

print("Configuração do Trainer concluída.")

In [ ]:
print("Iniciando o fine-tuning... Isso pode levar um bom tempo.")
print("Pegue um café ☕")

# Inicia o processo de treinamento:
# - Faz forward + backward pass
# - Atualiza pesos do modelo
# - Realiza avaliações periódicas (se configurado)
# - Salva checkpoints no diretório de saída
trainer.train()

print("🎉 Treinamento concluído! 🎉")

In [ ]:
print(f"Salvando o modelo final em '{PASTA_SAIDA}'...")

# Salva o modelo já fine-tunado no diretório especificado.
# Isso inclui:
# - pesos (pytorch_model.bin)
# - configuração do modelo (config.json)
# - estados de treinamento, se existirem (optimizer, scheduler)
trainer.save_model()

# Salva também o tokenizador utilizado no treinamento.
# Isso garante que, ao carregar o modelo no futuro,
# o texto será tokenizado da mesma forma.
tokenizer.save_pretrained(PASTA_SAIDA)

print(f"Modelo e Tokenizador salvos em '{PASTA_SAIDA}'.")

In [ ]:
# Instala bibliotecas adicionais necessárias para avaliação e manipulação de texto

!pip install bert_score              # Biblioteca para calcular BERTScore — métrica baseada em embeddings
                                     # que avalia similaridade semântica entre textos gerados e referência

!pip install sentence_transformers   # Fornece modelos pré-treinados para gerar embeddings de sentenças
                                     # (ex: SBERT) — útil para medir similaridade entre frases

!pip install nltk                    # Natural Language Toolkit: biblioteca clássica para NLP
                                     # usada para tokenização, stemming, métricas de texto etc.

!pip install pandas                  # Biblioteca para manipulação tabular de dados — útil para análise
                                     # e logging de métricas, experimentos, outputs etc.

!pip install google-generativeai     # Cliente oficial para acessar modelos generativos da Google (Gemini)
                                     # pode ser usado para comparar outputs com outros LLMs

In [ ]:
import torch
import pandas as pd
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from sentence_transformers import SentenceTransformer, util
from bert_score import score as bert_score_calc
from nltk.util import ngrams
import google.generativeai as genai
import random
import warnings

# Evita spam no console com avisos de bibliotecas
warnings.filterwarnings("ignore")


# Caminho do modelo fine-tunado a ser auditado
CAMINHO_MODELO = "./modelo_gpt2"

# Corpus de referência — usado para BERTScore, embeddings e perplexidade
ARQUIVO_CORPUS = "corpus.txt"

# Detecta GPU automaticamente, senão cai no CPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Chave opcional para avaliar textos via Gemini (LLM externo)
GEMINI_API_KEY = "COLE_SUA_CHAVE_API_AQUI"


class AvaliadorClarice:
    def __init__(self, modelo_path, corpus_path):

        # Carrega tokenizador e modelo GPT-2 fine-tunado
        self.tokenizer = GPT2Tokenizer.from_pretrained(modelo_path)
        self.model = GPT2LMHeadModel.from_pretrained(modelo_path).to(DEVICE)
        self.model.eval()  # Modo de inferência

        # Carrega modelo de embeddings (multi-língue, leve e eficiente)
        self.embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

        # Lê corpus original (usado como referência literária)
        with open(corpus_path, "r", encoding="utf-8") as f:
            self.raw_corpus = f.read()

        # Divide o corpus em blocos / parágrafos para comparação
        self.real_docs = [p for p in self.raw_corpus.split('\n\n') if len(p) > 50]


    def gerar_amostras(self, n_amostras=20):
        """Gera textos do modelo para avaliação posterior."""

        # Prompts iniciais estilizados — inspirados no estilo da Clarice
        prompts = ["O silêncio", "A vida", "Eu não", "O amor", "De repente", "A janela", "Senti que"]
        textos_gerados = []

        for _ in range(n_amostras):
            prompt = random.choice(prompts)
            inputs = self.tokenizer(prompt, return_tensors="pt").to(DEVICE)

            # Geração com sampling (não determinística)
            with torch.no_grad():
                out = self.model.generate(
                    **inputs,
                    max_length=150,
                    do_sample=True,
                    temperature=1.1,
                    repetition_penalty=1.0,
                    top_k=40,
                    top_p=0.95,
                    pad_token_id=self.tokenizer.eos_token_id
                )

            # Remove tokens especiais e reconverte para texto
            texto = self.tokenizer.decode(out[0], skip_special_tokens=True)
            textos_gerados.append(texto)

        return textos_gerados


    def calc_perplexity(self, texto):
        """Retorna PPL — quanto menor, mais o modelo "entende" esse texto."""

        encodings = self.tokenizer(texto, return_tensors="pt")
        max_len = self.model.config.n_positions
        stride = 512
        seq_len = encodings.input_ids.size(1)
        nlls = []
        prev_end_loc = 0

        # Calcula NLL em janelas móveis (evita limite de contexto)
        for begin_loc in range(0, seq_len, stride):
            end_loc = min(begin_loc + max_len, seq_len)
            trg_len = end_loc - prev_end_loc

            input_ids = encodings.input_ids[:, begin_loc:end_loc].to(DEVICE)
            target_ids = input_ids.clone()
            target_ids[:, :-trg_len] = -100  # Ignora tokens fora do trecho

            with torch.no_grad():
                outputs = self.model(input_ids, labels=target_ids)
                nlls.append(outputs.loss)

            prev_end_loc = end_loc

            if end_loc == seq_len: break

        return torch.exp(torch.stack(nlls).mean()).item()


    def calc_novelty(self, gerados, n=5):
        """
        Mede o quanto o texto é 'novo'.
        1.0 = totalmente novo
        0.0 = cópia literal
        """
        tokens_corpus = self.raw_corpus.split()
        corpus_ngrams = set(ngrams(tokens_corpus, n))

        scores = []
        for texto in gerados:
            tokens_gen = texto.split()
            if len(tokens_gen) < n: continue

            gen_ngrams = list(ngrams(tokens_gen, n))
            novos = sum(1 for ng in gen_ngrams if ng not in corpus_ngrams)

            ratio = novos / len(gen_ngrams)
            scores.append(ratio)

        return np.mean(scores)


    def calc_embeddings_distance(self, gerados):
        """
        Compara embeddings entre trechos gerados
        e trechos reais da Clarice.
        Alto = mais parecido
        """
        refs = random.sample(self.real_docs, min(len(gerados), len(self.real_docs)))

        emb_real = self.embedder.encode(refs, convert_to_tensor=True)
        emb_gen = self.embedder.encode(gerados, convert_to_tensor=True)

        cosine_scores = util.cos_sim(emb_gen, emb_real)
        return torch.mean(cosine_scores).item()


    def calc_bertscore(self, gerados):
        """
        Calcula Precision/Recall/F1 semântico usando BERT.
        Compara cada texto gerado com uma referência aleatória da Clarice.
        """
        refs = random.sample(self.real_docs, len(gerados))

        # Lang="pt" baixa o modelo BERTimbau ou similar automaticamente
        P, R, F1 = bert_score_calc(gerados, refs, lang="pt", verbose=False)
        return F1.mean().item()


    def llm_judge(self, gerados):
        """
        Pede para o Gemini agir como crítico literário.
        Útil quando queremos feedback qualitativo.
        """

        if not GEMINI_API_KEY:
            return "N/A"

        genai.configure(api_key=GEMINI_API_KEY)
        try:
            model_judge = genai.GenerativeModel('gemini-2.5-flash')
        except:
            return "Erro Modelo"

        texto_para_avaliar = gerados[0]

        prompt = f"""
        Atue como um crítico literário especialista em Clarice Lispector.
        Avalie o texto abaixo gerado por uma IA.
        [...]

        Responda APENAS com o número da nota.
        """

        try:
            response = model_judge.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            return f"Erro API: {e}"


    def executar_auditoria(self):

        # 1 — Avalia perplexidade no corpus real
        amostra_validacao = self.raw_corpus[:10000]
        ppl = self.calc_perplexity(amostra_validacao)

        # 2 — Gera textos novos
        gerados = self.gerar_amostras(n_amostras=15)

        # 3 — Métricas diversas
        bert_f1 = self.calc_bertscore(gerados)

        emb_sim = self.calc_embeddings_distance(gerados)

        novelty = self.calc_novelty(gerados, n=5)

        nota_llm = self.llm_judge(gerados)

        # 4 — Tabela final
        dados = {
            "Métrica": [
                "Perplexity",
                "Novelty",
                "BERTScore",
                "Embeddings",
                "LLM Judge"
            ],
            "Valor": [
                f"{ppl:.2f}",
                f"{novelty:.2%}",
                f"{bert_f1:.4f}",
                f"{emb_sim:.4f}",
                nota_llm
            ],
            "Interpretação Ideal": [
                "Baixo (< 30)",
                "Alto (> 90%) mas < 100%",
                "Alto (> 0.7)",
                "Alto (> 0.6)",
                "Alto (> 8.0)"
            ]
        }

        df = pd.DataFrame(dados)
        return df, gerados[0]


avaliador = AvaliadorClarice(CAMINHO_MODELO, ARQUIVO_CORPUS)
df_resultado, exemplo_texto = avaliador.executar_auditoria()

print(f"Amostra gerada: '{exemplo_texto[:100]}...'\n")

# No Colab, o display() imprime como tabela HTML
display(df_resultado)